# Technician Routing and Scheduling Problem

**Note**:
 
This notebook, originally created for opti201 optimization training, is almost identical to Gurobi's [`technician_routing_scheduling`](
https://colab.research.google.com/github/Gurobi/modeling-examples/blob/master/technician_routing_scheduling/technician_routing_scheduling.ipynb).
Since an additional post processing step was required to export the results into csv files, and also some slight modifications in the constraints and objective functions needed to be made, a slightly different file name is used to differentiate between the two.
 
To stay true to the original notebook, the changes are only made were absolutely necessary.

## Objective and Prerequisites

Try this modeling example to discover how mathematical optimization can help telecommunications firms automate and improve their technician assignment, scheduling, and routing decisions in order to ensure the highest levels of customer satisfaction.

This modeling example is at the intermediate level, where we assume that you know Python and are familiar with the Gurobi Python API. In addition, you have some knowledge about building mathematical optimization models. To fully understand the content of this notebook, you should be familiar with object-oriented-programming.

**Download the Repository** <br /> 
You can download the repository containing this and other examples by clicking [here](https://github.com/Gurobi/modeling-examples/archive/master.zip). 

**Gurobi License** <br /> 
In order to run this Jupyter Notebook properly, you must have a Gurobi license. If you do not have one, you can request an [evaluation license](https://www.gurobi.com/downloads/request-an-evaluation-license/?utm_source=3PW&utm_medium=OT&utm_campaign=WW-MU-TRA-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_technician-routing-sche-problem_COM_EVAL_GITHUB_&utm_term=technician-routing-scheduling-problem&utm_content=C_JPM) as a *commercial user*, or download a [free license](https://www.gurobi.com/academia/academic-program-and-licenses/?utm_source=3PW&utm_medium=OT&utm_campaign=WW-MU-TRA-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_technician-routing-sch-problem_ACADEMIC_EVAL_GITHUB_&utm_term=technician-routing-scheduling-problem&utm_content=C_JPM) as an *academic user*.

## Motivation
The Technician Routing and Scheduling (TRS) is a common problem faced by telecommunication firms, which must be able to provide services for a large number of customers. With its limited resources, these firms must try to deliver timely, affordable and reliable services to maximize customer satisfaction. The TRS problem involves assignment, scheduling, and routing of multi-skilled technicians to serve customers with unique priorities, service time windows, processing times, skills requirements, and geographical locations. 
In the current practice, these decisions are often being made by operators via heuristics, sometimes manually -and this method
can not only be time-consuming, 
but may also deviate from optima significantly. 
Using mathematical optimization to solve the
TRS problem  will
enable 
telecom operators and dispatchers to automate and improve their assignment, scheduling, and routing decisions, while achieving high customer satisfaction.

## Problem Description
A telecom firm operates multiple service centers to serve its customers. 
Each service center has 
its technicians who are dispatched from the service center to work on their assigned jobs, and return to the center after all the jobs are completed. 
A technician has multiple skills and available working capacity that cannot be exceeded during the scheduling horizon. 
A service order/job has 
a known processing time, 
a customer-specified time window for starting the service, 
a deadline of completing the service, 
and its skill requirements. 
Depending on the nature of the job (routine maintenance or emergency), and relationship with the customer (existing, new), the firm assesses the importance of the job and assigns a priority score to it. 
A job is assigned to at most one technician who possesses the required 
skills, 
but the technician's available capacity during the scheduling horizon cannot be exceeded.
To solve the  TRS problem, telecommunications companies must be able to
simultaneously make three types of decisions: (i) the assignment of jobs to a technician at all the service centers; 
(ii) the routing of each technician, i.e. the sequence/order of customers for a technician to visit; 
(iii) and the earliest and latest scheduling of jobs, i.e. the earliest and latest time for a technician to arrive at a customer location and complete the corresponding job. 
The firm’s goal is to maximize the number of jobs completed.

The following constraints must be satisfied:
* A technician, if utilized, departs from the service center where he/she is based and returns to the same service center after his/her assigned jobs are completed.
* A technician’s available capacity during the scheduling horizon cannot be exceeded.
* A job, if selected, is assigned to at most one technician who  possesses the required skills.
* A technician must arrive at a customer location during an interval (time window) specified by the customer, and must complete a job before the deadline required by the customer. This is an important constraint for guaranteeing customer satisfaction. 

The above basic TRS is a variant of the multi-depot vehicle routing problem with time windows, known as the MDVRPTW in the literature [1]. 


## Solution Approach

Mathematical programming is a declarative approach where the modeler formulates a mathematical optimization model that captures the key aspects of a complex decision problem. The Gurobi Optimizer solves such models using state-of-the-art mathematics and computer science.

A mathematical optimization model has five components, namely:

* Sets and indices.
* Parameters.
* Decision variables.
* Objective function(s).
* Constraints.

To simplify the MIP formulation, we consider that for each job's skill requirements, we can determine the subset of technicians that are qualified to perform the job based on the skills that the technicians have.

We now present a MIP formulation for the basic TRS problem.

## Model Formulation


### Sets and Indices
$j \in J = \{1,2,...,n\}$: Index and set of jobs.

$k \in K$: Index and set of technicians.

$K(j) \subset K$: Subset of technicians qualified to perform job $j \in J$ 

$d \in D = \{n+1, n+2, ...,n+m \}$: Index and set of depots (service centers), where $m$ is the number of depots.

$l, i, j \in L = J \cup D = \{1,2,..., n+m \}$: index and set of locations to visit.

### Parameters
$\beta_{k,d} \in \{0,1\}$: This parameter is equal 1 if technician $k \in K$ must depart from and return to depot 
$d \in D$; and 0 otherwise.

$p_{j} \in \mathbb{R^{+}}$; Processing time (duration) of job $j \in J$.

$\tau_{i,j} \in \mathbb{R^{+}}$: Travel time, in minutes, from location $i \in L$ to location $j \in L$.

$W_{k} \in \mathbb{N}$: Workload limit of technician $k \in K$ during the planning horizon. That is, the number of hours, in minutes, that the technician is available for the next work day.

$\pi_{j} \in \{1,2,3,4\}$: Priority weight of job $j \in J$, a larger number means higher priority.

$a_{j} \in \mathbb{R^{+}} $: Earliest time to start the service at location of job $j \in J$.

$b_{j} \in \mathbb{R^{+}} $: Latest time to start the service at location of job $j \in J$.

$M \in \mathbb{N}$: This is a very large number. This number is determined as follows: the planning horizon is 10 working hours (i.e. 600 min). We want the value of $M$ to be an order of magnitude larger than the planning horizon. Therefore, we set M = 6100.

### Decision Variables

$x_{j,k} \in \{0,1\}$: This variable is equal 1 if job $j \in J$ is assigned to technician $k \in K$, and 0 otherwise.

$u_{k} \in \{0,1\}$: This variable is equal 1 if technician $k \in K$  is used to perform a job, and 0 otherwise.

$y_{i,j,k} \in \{0,1\}$: This variable is equal 1 if technician $k \in K$  travels from location $i \in L$ to location 
$j \in L$; and 0 otherwise.

$t_{j} \geq 0$: This variable determines the time to arrive or start the service at location $j \in J$.

$g_{j}$: This variable is equal 1 if job $j \in J$ cannot be filled, and 0 otherwise. 

### Objective Function

- **Minimize Unfilled Jobs:** The Objective function is to minimize the number of jobs that could not be completed on time. We're also interested to get the earliest arrival time at each location. So, we minimize the time of arrival at each location as well.

\begin{equation}
\sum_{j \in J} \pi_{j} \cdot M \cdot g_{j} + \sum_{j \in J} 0.01 \cdot M \cdot t_{j}
\tag{0}
\end{equation}

**Note**: We want to treat the constraints about starting the jobs within a time window as hard constraints. Hard constraints can not be violated. We treat the constraints of filling the jobs demand as soft constraints. Soft constraints can be violated, but the violation will incur in a huge penalty. We assume that not filling jobs demand greatly deteriorate customer satisfaction, consequently should incur large penalties. Recall that the value of the parameter $M$ is a large number, then the penalty associated with not filling a job is determined as follows: $\pi_{j} \cdot M$.

To get the earliest arrival times of technicians at customer locations, we also add a term $0.01 \cdot M \cdot t_{j}$.


### Constraints

- **Assign qualified technicians:** For each job, we  assign one technician who is qualified for the job, or a gap is declared. 

\begin{equation}
\sum_{k \in K(j)} x_{j,k} + g_{j} = 1 \quad \forall j \in J
\tag{1}
\end{equation}

**Note**: The penalty of the gap variable $g_{j}$ is ($\pi_{j} \cdot M$) which is a large number to discourage 
not being able to satisfy demand.

- **Only one technician:** For each job, we only allow one technician to be assigned.

\begin{equation}
\sum_{k \in K} x_{j,k} \leq 1 \quad \forall j \in J
\tag{2}
\end{equation}

- **Technician capacity:** For each technician, we ensure that the available capacity of the technician is not exceeded. 

\begin{equation}
\sum_{j \in J}  p_{j} \cdot x_{j,k} + \sum_{i \in L} \sum_{j \in L} \tau_{i,j} \cdot y_{i,j,k} \leq W_{k} \cdot u_{k} \quad \forall k \in K
\tag{3}
\end{equation}

- **Technician tour:** For each technician and job, we ensure that if the technician is assigned to the job, then the technician must travel to another location (to form a tour). 

\begin{equation}
\sum_{j \in L}  y_{i,j,k} = x_{i,k} \quad \forall i \in J, k \in K
\tag{4}
\end{equation}

- For each technician and job, we ensure that if a technician is assigned to the job, then the technician must travel from another location to the location of the job (to form a tour). 

\begin{equation}
\sum_{i \in L}  y_{i,j,k} = x_{j,k} \quad \forall j \in J, k \in K
\tag{5}
\end{equation}

- **Same depot:** For each technician and depot, we ensure that a technician, if assigned to any job, must depart from and return to the service center (depot) where the technician is based. 

\begin{equation}
\sum_{j \in J}  y_{d,j,k} = \beta_{k,d} \cdot u_{k} \quad \forall  k \in K, d \in D
\tag{6}
\end{equation}

\begin{equation}
\sum_{i \in J}  y_{i,d,k} = \beta_{k,d} \cdot u_{k} \quad \forall  k \in K, d \in D
\tag{7}
\end{equation}

- **Temporal relationship:** For each location and job, we ensure the temporal relationship between two consecutive jobs served by the same technician. That is, if a technician $k$ travels from job $i$ to job $j$, then the start of the service time at job $j$ must be no less than the completion time of job $i$ plus the travel time from job $i$ to job $j$. 

\begin{equation}
t_{j} \geq t_{i} + p_{i} + \tau_{i,j} - M \cdot (1 - \sum_{k \in K}  y_{i,j,k}) \quad \forall i \in L, j \in J
\tag{8}
\end{equation}

**Note**: Observe that if the technician $k$ travels from the location of job $i$ to the location of job $j$, then
$\sum_{k \in K}  y_{i,j,k} = 1$. Therefore, $M \cdot (1 - \sum_{k \in K}  y_{i,j,k}) = 0$, and the constraint 
$t_{j} \geq t_{i} + p_{i} + \tau_{i,j}$ would be properly enforced. Now consider the case where the technician $k$ does not travel  from the location of job $i$ to the location of job $j$. Hence, $\sum_{k \in K}  y_{i,j,k} = 0$ and 
$M \cdot (1 - \sum_{k \in K}  y_{i,j,k}) = M$. In this case, this constraint becomes 
$t_{j} \geq t_{i} + p_{i} + \tau_{i,j} - M$. But $M$ is a very large number, then $t_{i} + p_{i} + \tau_{i,j} - M < 0$ and 
since $t_{j} \geq 0$, this constraint is redundant.

- **Time window:** For each job $j \in J$ ensure that the time window for the job is satisfied.

\begin{equation}
t_{j} \geq a_{j} \quad \forall j \in J
\tag{9}
\end{equation}

\begin{equation}
t_{j} \leq b_{j} \quad \forall j \in J
\tag{10}
\end{equation}

## Problem Instance

In this scenario, we consider the problem of routing and scheduling technicians for the next work-day in such a way that the delay of customer appointments is minimized. The telecom firm has seven technicians: Albert, Bob, Carlos, Doris, Ed, Flor, and Gina. There are two service centers: Heidelberg and Freiburg im Breisgau. Technicians are based at only one of these service centers. The number of hours available and the service center base (depot) for each technician is described in the following table.

| <i></i> | Albert | Bob | Carlos | Doris | Ed | Flor | Gina |
| --- | --- | --- | --- | --- | --- | --- | --- |
| Minutes | 480 | 480 | 480 | 480 | 480 | 360 | 360 |
| Depot | Heidelberg | Heidelberg | Freiburg im Breisgau | Freiburg im Breisgau | Heidelberg | Freiburg im Breisgau | Heidelberg |

The telecom company has different type of jobs. 
The following table shows the priority (4 for the highest priority and 1 for the least important) and duration (in hours) of a job type.

| <i></i> | Priority | Duration (min) |
| --- | --- | --- | 
| Equipment Installation | 2 | 60 |
| Equipment Setup | 3 | 30 |
| Inspect/Service Equipment | 1 | 60 |
| Repair - Regular | 1 | 60 |
| Repair - Important | 2 | 120 |
| Repair - Urgent | 3 | 90 |
| Repair - Critical | 4 | 60 |

The following table shows the jobs that each technician is qualified for.

| <i></i> | Albert | Bob | Carlos | Doris | Ed | Flor | Gina |
| --- | --- | --- | --- | --- | --- | --- | --- |
| Equipment Installation | - | - | - | 1 | - | 1 | 1 |
| Equipment Setup | 1 | 1 | 1 | 1 | - | - | 1 |
| Inspect/Service Equipment | 1 | - | 1 | - | 1 | - | - |
| Repair - Regular | - | 1 | 1 | - | 1 | 1 | 1 |
| Repair - Important | - | - | - | 1 | - | 1 | 1 |
| Repair - Urgent | - | 1 | 1 | - | 1 | 1 | 1 |
| Repair - Critical | - | - | - | 1 | - | - | 1 |

The telecom company receives customers' requests for a specific job type, appointment (due) time, and the service time window where the technician can arrive. The following table shows the customers' orders and their requirements. For each customer, the location is specified.

| <i></i> | C1:Mannheim  | C2: Karlsruhe  | C3: Baden-Baden  | C4: Bühl  | C5: Offenburg | C6: Lahr/Schwarzwald | C7: Lörrach |
| --- | --- | --- | --- | --- | --- | --- | --- |
| Job type | Equipment Setup | Equipment Setup | Repair - Regular | Equipment Installation | Equipment Installation | Repair - Critical | Inspect/Service Equipment |
| Due time | 8:00 | 10:00 | 11:00  | 12:00 | 14:00 | 15:00 | 16:00 |
| Time Window | 7:00-7:30 | 7:30-9:30 | 8:00-10:00 | 9:00-11:00 | 11:00-13:00 | 12:00-14:00 | 13:00-15:00 |

The planning horizon is from 7:00 to 17:00, or 10 hours. The time period is in minutes, then the due time and time windows will be translated into minutes starting at 0 minutes and ending at 600 minutes. For example, for customer C2 the due time is at 10:00  (180 min), and the time window is from 7:30 to 9:30  (30 min to  150 min). 

The following table shows the travel time (in minutes) from any depot or customer location to any depot or customer location.

| <i></i> | Heidelberg | Freiburg im Breisgau | Mannheim  | Karlsruhe  | Baden-Baden  | Bühl  | Offenburg | Lahr/Schwarzwald | Lörrach |
| --- | --- | --- | --- | --- | --- | --- | --- |--- |--- |
| Heidelberg | - | 120 | 24 | 50 | 67 | 71 | 88 | 98 | 150 |
| Freiburg im Breisgau | - | - | 125 | 85 | 68 | 62 | 45 | 39 | 48 |
| Mannheim | - | - | - | 53 | 74 | 77 | 95 | 106 | 160 |
| Karlsruhe | - | - | - | - | 31 | 35 | 51 | 61 | 115 |
| Baden-Baden | - | - | - | - | - | 16 | 36 | 46 | 98 |
| Bühl | - | - | - | - | - | - | 30 | 40 | 92 |
| Offenburg | - | - | - | - | - | - | - | 26 | 80 |
| Lahr/Schwarzwald | - | - | - | - | - | - | - | - | 70 |
| Lörrach | - | - | - | - | - | - | - | - | - |


## Python Implementation

In this Jupyter Notebook, we use the following libraries:

* `sys` to access system-specific parameters and functions.
* `pandas` to read data from Excel files.
* `gurobipy` Gurobi Optimizer library.

This implementation is based on object-oriented-programming.

### Helper Classes

The following classes properly organize the input data of the MIP model.

* `Technician`.
* `Job`.
* `Customer`.

### Helper Functions

* `solve_trs0` builds and solves MIP model.
* `get_latest_times` solves for latest arrival times at customer locations, given the solution from the MIP Model.
* `printScen` prints headings for output reports.


In [ ]:
%pip install gurobipy
%pip install pandas

In [1]:
import sys
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# tested with Gurobi v10.0.3

In [2]:
class Technician():
    def __init__(self, name, cap, depot):
        self.name = name
        self.cap = cap
        self.depot = depot

    def __str__(self):
        return f"Technician: {self.name}\n  Capacity: {self.cap}\n  Depot: {self.depot}"

In [3]:
class Job():
    def __init__(self, name, priority, duration, coveredBy):
        self.name = name
        self.priority = priority
        self.duration = duration
        self.coveredBy = coveredBy

    def __str__(self):
        about = f"Job: {self.name}\n  Priority: {self.priority}\n  Duration: {self.duration}\n  Covered by: "
        about += ", ".join([t.name for t in self.coveredBy])
        return about

In [4]:
class Customer():
    def __init__(self, name, loc, job, tStart, tEnd, tDue):
        self.name = name
        self.loc = loc
        self.job = job
        self.tStart = tStart
        self.tEnd = tEnd
        self.tDue = tDue

    def __str__(self):
        coveredBy = ", ".join([t.name for t in self.job.coveredBy])
        return f"Customer: {self.name}\n  Location: {self.loc}\n  Job: {self.job.name}\n  Priority: {self.job.priority}\n  Duration: {self.job.duration}\n  Covered by: {coveredBy}\n  Start time: {self.tStart}\n  End time: {self.tEnd}\n  Due time: {self.tDue}"


## Scenario

For this scenario, there is enough technician capacity to fill all customers demand. To run the base scenario, consider the spreadsheet file `data-Sce0.xlsx` and insert this name when you open the Excel workbook.

In [5]:
# Read Excel workbook
excel_file = 'https://raw.githubusercontent.com/decision-spot/technician_assignment/main/data-Sce0.xlsx'
df = pd.read_excel(excel_file, sheet_name='Technicians')
df = df.rename(columns={df.columns[0]: "name", df.columns[1]: "cap", df.columns[2]: "depot"})

df1 = df.drop(df.columns[3:], axis=1).drop(df.index[[0,1]])
# Create Technician objects
technicians = [Technician(*row) for row in df1.itertuples(index=False, name=None)]
# print(df1)

In [6]:
# Read job data
jobs=[]
for j in range(3, len(df.columns)):
    coveredBy = [t for i, t in enumerate(technicians) if df.iloc[2+i,j]==1]
    thisJob = Job(df.iloc[2:,j].name, df.iloc[0,j], df.iloc[1,j], coveredBy)
    jobs.append(thisJob)

In [7]:
# Read location data
df_locations = pd.read_excel(excel_file, sheet_name='Locations', index_col=0) #, skiprows=1, index_col=0)

# Extract locations and initialize distance dictionary
locations = df_locations.index
dist = {(l, l): 0 for l in locations}

# Populate distance dictionary
for i, l1 in enumerate(locations):
    for j, l2 in enumerate(locations):
        if i < j:
            dist[l1, l2] = df_locations.iloc[i, j]
            dist[l2, l1] = dist[l1, l2]

In [8]:
# Read customer data
df_customers = pd.read_excel(excel_file, sheet_name='Customers')

customers = []
for i, c in enumerate(df_customers.iloc[:, 0]):
    job_name = df_customers.iloc[i, 2]

    # Find the corresponding Job object
    matching_job = next((job for job in jobs if job.name == job_name), None)

    if matching_job is not None:
        # Create Customer object using corresponding Job object
        this_customer = Customer(c, df_customers.iloc[i, 1], matching_job, *df_customers.iloc[i, 3:])
        customers.append(this_customer)

To determine the latest times for a technician to arrive at a customer location and complete the job, you can calculate it by iterating over the sequence of customers to be served by the technician in reverse order.

The latest time a technician can arrive at a customer location is constrained by two factors. First, it is upper-bounded by the customer-specified time window's End Time.  
Second, it is also limited by the following: the latest arrival time at the next customer location in the sequence minus the job duration at the current customer location and the distance between the two customer locations. Additionally, it's essential for the technician to reach the depot location after completing all the assigned jobs by time t = 600.

In [6]:
def get_latest_times(customers, technician):
    latest = dict()
    d = dist[customers[-1].loc, technician.depot]   # distance back to the depot
    prev_latest = min(customers[-1].tEnd, 600 - d - customers[-1].job.duration)
    latest[customers[-1].loc] = prev_latest
    for i in range(len(customers) - 2, -1, -1):
        d = dist[customers[i].loc, customers[i + 1].loc]
        latest_end = min(prev_latest - d - customers[i].job.duration, customers[i].tEnd)
        latest[customers[i].loc] = latest_end
        prev_latest = latest_end
    return latest

We added another function `create_excel_output` inside the `solve_trs0` function to store the solution of the problem in two tables: **routes** and **orders**.

In [7]:
def solve_trs0(technicians, customers, dist):
    # Build useful data structures
    K = [k.name for k in technicians]
    C = [j.name for j in customers]
    J = [j.loc for j in customers]
    L = list(set([l[0] for l in dist.keys()]))
    D = list(set([t.depot for t in technicians]))
    cap = {k.name: k.cap for k in technicians}
    loc = {j.name: j.loc for j in customers}
    depot = {k.name: k.depot for k in technicians}
    canCover = {j.name: [k.name for k in j.job.coveredBy] for j in customers}
    dur = {j.name: j.job.duration for j in customers}
    tStart = {j.name: j.tStart for j in customers}
    tEnd = {j.name: j.tEnd for j in customers}
    priority = {j.name: j.job.priority for j in customers}

    ### Create model
    m = gp.Model("trs0")

    ### Decision variables
    # Customer-technician assignment
    x = m.addVars(C, K, vtype=GRB.BINARY, name="x")

    # Technician assignment
    u = m.addVars(K, vtype=GRB.BINARY, name="u")

    # Edge-route assignment to technician
    y = m.addVars(L, L, K, vtype=GRB.BINARY, name="y")

    # Technician cannot leave or return to a depot that is not its base
    for k in technicians:
        for d in D:
            if k.depot != d:
                for i in L:
                    y[i, d, k.name].ub = 0
                    y[d, i, k.name].ub = 0

    # Start time of service
    t = m.addVars(L, ub=600, name="t")

    # Unfilled jobs
    g = m.addVars(C, vtype=GRB.BINARY, name="g")

    ### Constraints

    # A technician must be assigned to a job, or a gap is declared (1)
    m.addConstrs((gp.quicksum(x[j, k] for k in canCover[j]) + g[j] == 1 for j in C), name="assignToJob")

    # At most one technician can be assigned to a job (2)
    m.addConstrs((x.sum(j, '*') <= 1 for j in C), name="assignOne")

    # Technician capacity constraints (3)
    capLHS = {k: gp.quicksum(dur[j] * x[j, k] for j in C) + \
                 gp.quicksum(dist[i, j] * y[i, j, k] for i in L for j in L) for k in K}
    m.addConstrs((capLHS[k] <= cap[k] * u[k] for k in K), name="techCapacity")

    # Technician tour constraints (4 and 5)
    m.addConstrs((y.sum('*', loc[j], k) == x[j, k] for k in K for j in C), \
                 name="techTour1")
    m.addConstrs((y.sum(loc[j], '*', k) == x[j, k] for k in K for j in C), \
                 name="techTour2")

    # Same depot constraints (6 and 7)
    m.addConstrs((gp.quicksum(y[j, depot[k], k] for j in J) == u[k] for k in K), \
                 name="sameDepot1")
    m.addConstrs((gp.quicksum(y[depot[k], j, k] for j in J) == u[k] for k in K), \
                 name="sameDepot2")

    # Temporal constraints (8) for customer locations
    M = {(i, j): 600 + dur[i] + dist[loc[i], loc[j]] for i in C for j in C}
    m.addConstrs((t[loc[j]] >= t[loc[i]] + dur[i] + dist[loc[i], loc[j]] \
                  - M[i, j] * (1 - gp.quicksum(y[loc[i], loc[j], k] for k in K)) \
                  for i in C for j in C), name="tempoCustomer")

    # Temporal constraints (8) for depot locations
    M = {(i, j): 600 + dist[i, loc[j]] for i in D for j in C}
    m.addConstrs((t[loc[j]] >= t[i] + dist[i, loc[j]] \
                  - M[i, j] * (1 - y.sum(i, loc[j], '*')) for i in D for j in C), \
                 name="tempoDepot")

    # Time window constraints (9 and 10)
    m.addConstrs((t[loc[j]] >= tStart[j] for j in C), name="timeWinA")
    m.addConstrs((t[loc[j]] <= tEnd[j] for j in C), name="timeWinB")

    ### Objective function
    M = 6100

    m.setObjective(gp.quicksum(M * priority[j] * g[j] for j in C) + gp.quicksum(0.01 * M * t[k] for k in L),
                   GRB.MINIMIZE)

    m.write("TRS0.lp")
    m.optimize()

    status = m.Status
    if status in [GRB.INF_OR_UNBD, GRB.INFEASIBLE, GRB.UNBOUNDED]:
        print("Model is either infeasible or unbounded.")
        sys.exit(0)
    elif status != GRB.OPTIMAL:
        print("Optimization terminated with status {}".format(status))
        sys.exit(0)

    ### Print results
    # Assignments
    print("")
    for j in customers:
        if g[j.name].X > 0.5:
            jobStr = "Nobody assigned to {} ({}) in {}".format(j.name,j.job.name,j.loc)
        else:
            for k in K:
                if x[j.name, k].X > 0.5:
                    jobStr = f"{k} assigned to {j.name} ({j.job.name}) in {j.loc}. Start at t={t[j.loc].X:.2f}."
        print(jobStr)

    # Technicians
    print("")
    for k in technicians:
        if u[k.name].X > 0.5:
            cur = k.depot
            route = k.depot
            while True:
                for j in customers:
                    if y[cur,j.loc,k.name].X > 0.5:
                        route += (f" -> {j.loc} (dist={dist[cur, j.loc]}, t={t[j.loc].X:.2f},"
                                  f" proc={j.job.duration}, a={tStart[j.name]}, b={tEnd[j.name]})")
                        cur = j.loc
                for i in D:
                    if y[cur,i,k.name].X > 0.5:
                        route += " -> {} (dist={})".format(i,dist[cur, i])
                        cur = i
                        break
                if cur == k.depot:
                    break
            print("{}'s route: {}".format(k.name, route))
        else:
            print("{} is not used".format(k.name))

            
    # Utilization
    print("")
    for k in K:
        used = capLHS[k].getValue()
        total = cap[k]
        util = used / cap[k] if cap[k] > 0 else 0
        print("{}'s utilization is {:.2%} ({:.2f}/{:.2f})".format(k, util, used, cap[k]))
    totUsed = sum(capLHS[k].getValue() for k in K)
    totCap = sum(cap[k] for k in K)
    totUtil = totUsed / totCap if totCap > 0 else 0
    print("Total technician utilization is {:.2%} ({:.2f}/{:.2f})".format(totUtil, totUsed, totCap))
    
    def create_excel_output():
        routes_cols = ['Route ID', 'Technician Name', 'Origin Location', 'Total Travel Time', 'Total Processing Time',
                       'Total Time', 'Earliest Start Time', 'Latest Start Time', 'Earliest End Time',
                       'Latest End Time', 'Num Jobs']
        routes_list = []
        orders_cols = ['Route ID', 'Stop Number', 'Customer Name', 'Technician Name', 'Location Name', 'Job type',
                       'Processing Time', 'Customer Time Window Start', 'Customer Time Window End',
                       'Earliest Start', 'Latest Start', 'Earliest End', 'Latest End', ]
        route_id = 0
        orders_list = []
        for k in technicians:
            if u[k.name].X > 0.5:
                customers_list = []
                route_id += 1
                total_distance, total_travel_time, total_processing_time = 0, 0, 0
                cur = k.depot
                while True:
                    for j in customers:
                        if y[cur, j.loc, k.name].X > 0.5:
                            total_travel_time += dist[cur, j.loc]
                            total_distance += dist[cur, j.loc]
                            total_processing_time += j.job.duration
                            customers_list.append(j)
                            cur = j.loc
                    for i in D:
                        if y[cur, i, k.name].X > 0.5:
                            total_travel_time += dist[cur, i]
                            total_distance += dist[cur, i]
                            cur = i
                            break
                    if cur == k.depot:
                        break
                latest = get_latest_times(customers_list, k)

                # append customers to the list of orders
                for i, j in enumerate(customers_list):
                    orders_list.append([route_id, i + 1, j.name, k.name, j.loc, j.job.name,
                                        j.job.duration, tStart[j.name], tEnd[j.name],
                                        t[j.loc].X, latest[j.loc],
                                        t[j.loc].X + j.job.duration, latest[j.loc] + j.job.duration])

                # append route to routes list
                earliest_start_route = t[customers_list[0].loc].X - dist[k.depot, customers_list[0].loc]
                latest_start_route = latest[customers_list[0].loc] - dist[k.depot, customers_list[0].loc]
                earliest_end_route = t[customers_list[-1].loc].X + customers_list[-1].job.duration + \
                                     dist[customers_list[-1].loc, k.depot]
                latest_end_route = latest[customers_list[-1].loc] + customers_list[-1].job.duration + \
                                   dist[customers_list[-1].loc, k.depot]

                routes_list.append([route_id, k.name, k.depot, total_travel_time, total_processing_time,
                                    earliest_end_route - earliest_start_route, earliest_start_route,
                                    latest_start_route, earliest_end_route, latest_end_route, len(customers_list)])
        # Convert to dataframe and write to excel
        routes_df = pd.DataFrame.from_records(routes_list, columns=routes_cols)
        routes_df.to_csv('routes.csv', index=False)
        orders_df = pd.DataFrame.from_records(orders_list, columns=orders_cols)
        orders_df.to_csv('orders.csv', index=False)
    
    # create output files
    create_excel_output()
    
    m.dispose()
    gp.disposeDefaultEnv()

In [8]:
def printScen(scenStr):
    sLen = len(scenStr)
    print("\n" + "*"*sLen + "\n" + scenStr + "\n" + "*"*sLen + "\n")

In [9]:
if __name__ == "__main__":
    # Base model
    printScen("Solving base scenario model")
    solve_trs0(technicians, customers, dist)


***************************
Solving base scenario model
***************************

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-19
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 210 rows, 639 columns and 2301 nonzeros
Model fingerprint: 0x5263eee2
Variable types: 9 continuous, 630 integer (630 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [6e+01, 2e+04]
  Bounds range     [1e+00, 6e+02]
  RHS range        [1e+00, 6e+02]
Found heuristic solution: objective 165310.00000
Presolve removed 87 rows and 395 columns
Presolve time: 0.02s
Presolved: 123 rows, 244 columns, 944 nonzeros
Variable types: 0 continuous, 244 integer (238 binary)

Root relaxation: objective 6.992552e+04, 72 iterations, 0.00 seconds 

## Scenario Analysis
For the base scenario, we have enough technician capacity to satisfy customers demand. Notice that  all jobs were completed and they were completed within time window limits.

The first report describes which technician was assigned to each customer, the location of the customer, and the start time of the job. The second report describes the route of each assigned technician. The route defines the *from-location* and the 
*to-location*. The parameter *dist*  defines the number of minutes that it takes to drive from the *from-location* to the *to-location*. The parameter t determines the *start-time* of the job at the *to-location*. The parameter *proc*  shows the number of minutes that it takes the technician to complete the job. 
The third report describes the capacity utilization of each technician and the overall capacity utilization of all technicians. The technician capacity utilization is the number of minutes the technician spends driving or serving customers, divided by the technician capacity. The overall capacity utilization of all technicians is the total number of minutes that all technicians spend driving or serving customers, divided by the total capacity of all the technicians available. 

**Note**:

With the current objective function, you can get very different results and all of them are correct. Because the objective function mainly focuses on minimizing unfilled demands. So, you can get a solution with 4 routes and another one with 6.

## Conclusion

In this Jupyter Notebook, we formulated and solved a multi-depot vehicle routing problem with time window constraints using the Gurobi Python API. This MIP formulation is an optimization application for the telecommunication industry; however, the formulation is quite general and can easily be adapted to any kind of vehicle routing problems in the transportation and logistics industries.

The routing and scheduling problem that we tackled considers a telecom firm that operates multiple service centers to serve its customers. We solved this routing and scheduling problem by simultaneously making three types of decisions: 
- (i) the assignment of jobs to a technician at all the service centers
- (ii) the routing of each technician, i.e. the sequence/order of customers for a technician to visit
- (iii) the scheduling of jobs, i.e. the earliest and latest arrival times for a technician to arrive at a customer location and complete the corresponding job. 

The objective of the telecom firm is to maximize the total number of completed jobs.

## References
[1] S. Salhi, A. Imran, N. A. Wassan. *The multi-depot vehicle routing problem with heterogeneous vehicle fleet: Formulation and a variable neighborhood search implementation*. Computers & Operations Research 52 (2014) 315-325.

Copyright © 2023 Gurobi Optimization, LLC